### Loading in the data



In [1]:
import matplotlib.pyplot as plt
#import statsmodels.tsa.seasonal as smt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import datetime as dt

In [4]:
import os
from os.path import expanduser
home = expanduser("~")
os.chdir(home+'\Desktop\Deep Learning Project\Data\price-volume-data-for-all-us-stocks-etfs\Data\Stocks')
os.getcwd()

'C:\\Users\\Thomas\\Desktop\\Deep Learning Project\\Data\\price-volume-data-for-all-us-stocks-etfs\\Data\\Stocks'

In [6]:
#read data
#takes 1 minute for 1000 files

filenames = [x for x in os.listdir() if x.endswith('.txt') and os.path.getsize(x) > 0]
#filenames = ['a.us.txt', aa.us.txt, 'bgr.us.txt']
filenames=filenames[:100]


#data = []
result=None
for filename in filenames:
    df = pd.read_csv(filename, sep=',')

    label, _, _ = filename.split(sep='.')
    df.insert(loc=0, column='Label', value=filename)
    df['Date'] = pd.to_datetime(df['Date'])
    df['ReturnOpenPrevious']=(df.Open-df.Open.shift(1))/df.Open.shift(1)
    df['ReturnOpenNext']=(df.Open.shift(-1)-df.Open)/df.Open
    #data.append(df)
    if type(result)==type(None):
        result=df
    else:
        result=result.append(df,ignore_index=True)
    
result.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenPrevious,ReturnOpenNext
0,a.us.txt,1999-11-18,30.713,33.754,27.002,29.702,66277506,0,NaN,-0.056230
1,a.us.txt,1999-11-19,28.986,29.027,26.872,27.257,16142920,0,-0.056230,-0.037949
2,a.us.txt,1999-11-22,27.886,29.702,27.044,29.702,6970266,0,-0.037949,0.028760
3,a.us.txt,1999-11-23,28.688,29.446,27.002,27.002,6332082,0,0.028760,-0.055947
4,a.us.txt,1999-11-24,27.083,28.309,27.002,27.717,5132147,0,-0.055947,0.018868


In [7]:
result1=result.copy()
encoder_Label = dict()
encoder_Label = {l: id for id, l in enumerate(filenames)}

result1.Label=[encoder_Label.get(i) for i in result1.Label]
result1.tail()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenPrevious,ReturnOpenNext
260881,99,2017-11-06,8.00,8.15,7.800,7.85,409212,0,0.059603,-0.012500
260882,99,2017-11-07,7.90,8.00,7.750,7.80,464754,0,-0.012500,-0.018987
260883,99,2017-11-08,7.75,7.90,7.525,7.80,392643,0,-0.018987,-0.006452
260884,99,2017-11-09,7.70,7.90,7.500,7.85,465090,0,-0.006452,0.025974
260885,99,2017-11-10,7.90,8.15,7.750,7.95,558278,0,0.025974,NaN


In [12]:
result2=result1.copy()
encoder_Date = dict()
encoder_Date = {l: id for id, l in enumerate(result2.Date.unique())}
result2.Date=[encoder_Date.get(i) for i in result2.Date.values]
result2.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenPrevious,ReturnOpenNext
0,0,0,30.713,33.754,27.002,29.702,66277506,0,NaN,-0.056230
1,0,1,28.986,29.027,26.872,27.257,16142920,0,-0.056230,-0.037949
2,0,2,27.886,29.702,27.044,29.702,6970266,0,-0.037949,0.028760
3,0,3,28.688,29.446,27.002,27.002,6332082,0,0.028760,-0.055947
4,0,4,27.083,28.309,27.002,27.717,5132147,0,-0.055947,0.018868


8

In [14]:
#result3=np.empty((len(encoder_Label),len(encoder_Date),6))
#result3[:,:,:]=None
#result3.shape
#for index, row in result2.iterrows():
#   result3[int(row['Label']),int(row['Date']),:]=row[['Open','High','Low','Close','Volume','OpenInt']]
#result3

result3=np.empty((len(encoder_Label),len(encoder_Date),result2.shape[1]-2))
result3[:,:,:]=None

for row in result2.itertuples():
    result3[int(row.Label),int(row.Date),:]=row[3:]
result3[0,0,:]


array([ 3.07130000e+01,  3.37540000e+01,  2.70020000e+01,  2.97020000e+01,
        6.62775060e+07,  0.00000000e+00,             nan, -5.62302608e-02])

In [792]:
result3.shape

(1000, 12077, 6)

### Implementaion Baseline Model
### Starting from result2 (Labels and Dates encoded

In [23]:
result2.head()


,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenPrevious,ReturnOpenNext
0,0,0,30.713,33.754,27.002,29.702,66277506,0,NaN,-0.056230
1,0,1,28.986,29.027,26.872,27.257,16142920,0,-0.056230,-0.037949
2,0,2,27.886,29.702,27.044,29.702,6970266,0,-0.037949,0.028760
3,0,3,28.688,29.446,27.002,27.002,6332082,0,0.028760,-0.055947
4,0,4,27.083,28.309,27.002,27.717,5132147,0,-0.055947,0.018868


In [79]:
market_df=result2.sort_values(['Date','Label'])
market_df.reset_index(inplace=True,drop=True)
market_df.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenPrevious,ReturnOpenNext
0,0,0,30.7130,33.7540,27.0020,29.7020,66277506,0,NaN,-0.056230
1,1,0,64.8500,64.8500,63.6410,64.0830,1057225,0,0.002861,0.001912
2,3,0,51.6600,53.8800,50.7500,53.4700,16055200,0,-0.021035,0.035230
3,12,0,2.9159,2.9171,2.8312,2.8698,101732076,0,0.004374,-0.017044
4,23,0,2.9374,3.2055,2.9193,3.1331,6184251,0,0.000000,0.066624


In [92]:
from sklearn.model_selection import train_test_split

market_train_indices, market_val_indices = train_test_split(market_df.index,test_size=0.25, random_state=23)

# Handling numerical variables

In [99]:
#from sklearn.preprocessing import StandardScaler
#market_df.columns.get_loc("ReturnOpenPrevious")


8

['',
 'import matplotlib.pyplot as plt\n#import statsmodels.tsa.seasonal as smt\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\nimport random\nimport datetime as dt',
 'import os\nfrom os.path import expanduser\nhome = expanduser("~")\nos.chdir(home+\'\\Desktop\\Deep Learning Project\\Data\\price-volume-data-for-all-us-stocks-etfs\\Data\\Stocks\')\nos.getcwd()',
 'import os\nfrom os.path import expanduser\nhome = expanduser("~")\nos.chdir(home+\'\\Desktop\\Deep Learning Project\\Data\\price-volume-data-for-all-us-stocks-etfs\\Data\\Stocks\')\nos.getcwd()',
 'import os\nfrom os.path import expanduser\nhome = expanduser("~")\nos.chdir(home+\'\\Desktop\\Deep Learning Project\\Data\\price-volume-data-for-all-us-stocks-etfs\\Data\\Stocks\')\nos.getcwd()',
 "#read data\n#takes 1 minute for 1000 files\n\nfilenames = [x for x in os.listdir() if x.endswith('.txt') and os.path.getsize(x) > 0]\n#filenames = ['a.us.txt', aa.us.txt, 'bgr